In [2]:
!git clone https://github.com/pondelion/FinAppModel.git

Cloning into 'FinAppModel'...
remote: Enumerating objects: 536, done.
remote: Counting objects: 100% (536/536), done.
remote: Compressing objects: 100% (307/307), done.
remote: Total 536 (delta 253), reused 444 (delta 167), pack-reused 0
Receiving objects: 100% (536/536), 100.31 KiB | 1.41 MiB/s, done.
Resolving deltas: 100% (253/253), done.


In [1]:
%cd FinAppModel

/content/FinAppModel


In [4]:
!python -V

Python 3.7.12


In [4]:
!ls

extra_install.sh  Pipfile	  README.md	    setup.py
fin_app_models	  pyproject.toml  requirements.txt  tests


In [5]:
!./extra_install.sh > /dev/null

--2022-01-06 05:50:24--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2022-01-06 05:50:25--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: http://nchc.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2022-01-06 05:50:25--  http://nchc.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving nchc.dl.sourceforge.n

In [6]:
!pip install -r requirements.txt

     |████████████████████████████████| 54 kB 1.8 MB/s 
     |████████████████████████████████| 1.1 MB 8.5 MB/s 
     |████████████████████████████████| 156 kB 56.8 MB/s 
     |████████████████████████████████| 49 kB 4.8 MB/s 
     |████████████████████████████████| 131 kB 48.9 MB/s 
     |████████████████████████████████| 8.5 MB 43.0 MB/s 
     |████████████████████████████████| 65.7 MB 46 kB/s 
     |████████████████████████████████| 80 kB 8.0 MB/s 
     |████████████████████████████████| 133 kB 45.3 MB/s 
     |████████████████████████████████| 2.0 MB 18.1 MB/s 
     |████████████████████████████████| 98 kB 7.9 MB/s 
     |████████████████████████████████| 58 kB 6.0 MB/s 
     |████████████████████████████████| 303 kB 55.8 MB/s 
     |████████████████████████████████| 1.1 MB 41.9 MB/s 
     |████████████████████████████████| 1.8 MB 40.8 MB/s 
     |████████████████████████████████| 75 kB 4.1 MB/s 
     |████████████████████████████████| 11.5 MB 22.6 MB/s 
     |█████████████████████

In [7]:
!pip install -e .

Obtaining file:///content/FinAppModel
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Running setup.py develop for fin-app-model


In [2]:
from glob import glob
from datetime import date
import pandas as pd
from fastprogress import progress_bar as pb

In [3]:
df_btc_return = pd.read_csv('/content/drive/MyDrive/data/crypto/historical/gmo/return_btc_5min.csv')
df_btc_return['timestamp'] = pd.to_datetime(df_btc_return['timestamp'])
df_btc_return.set_index('timestamp', inplace=True)

In [4]:
df_btc_return

,size_buy,price_buy,size_sell,price_sell,return_1h,return_3h,return_6h,return_12h,return_18h,return_24h,return_48h,return_72h,return_120h,return_168h,return_720h,return_1440h,return_2160h
timestamp,,,,,,,,,,,,,,,,,
2019-12-31 21:15:00,0.003475,777540.00,0.012000,7.775560e+05,1.000916,1.000649,1.006713,1.002986,1.007373,1.008283,0.968452,1.017735,1.029784,1.132307,1.336909,1.201652,0.905107
2019-12-31 21:20:00,0.001000,777580.00,0.018333,7.776577e+05,1.000900,1.000561,1.007009,1.003472,1.006925,1.008224,0.968900,1.016635,1.029214,1.130117,1.336218,1.199488,0.904578
2019-12-31 21:25:00,0.001367,777862.00,0.024667,7.777593e+05,1.000971,1.000161,1.007203,1.002903,1.006875,1.007851,0.969047,1.015219,1.027670,1.129068,1.335293,1.198884,0.905345
2019-12-31 21:30:00,0.001000,777862.00,0.031000,7.778610e+05,1.001404,1.000124,1.007762,1.003104,1.007004,1.007844,0.969545,1.014171,1.026500,1.126282,1.334464,1.199147,0.903535
2019-12-31 21:35:00,0.001000,777830.00,0.065500,7.776455e+05,1.001878,1.000129,1.007502,1.003347,1.007482,1.007878,0.970083,1.013165,1.025095,1.125580,1.332712,1.199459,0.903243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-01 20:30:00,0.000158,5463890.25,0.004437,5.463901e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-01 20:35:00,0.000100,5469121.00,0.005000,5.469120e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-01 20:40:00,0.003750,5465023.00,0.010000,5.462800e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
TARGET_COLS = [col for col in df_btc_return.columns if 'return' in col]

In [6]:
TARGET_COLS

['return_1h',
 'return_3h',
 'return_6h',
 'return_12h',
 'return_18h',
 'return_24h',
 'return_48h',
 'return_72h',
 'return_120h',
 'return_168h',
 'return_720h',
 'return_1440h',
 'return_2160h']

In [7]:
from fin_app_models.feature.creation.single_ts import create_single_ts_features
from fin_app_models.feature.selection.random_selection import random_feat_select

In [8]:
# 5分間隔データのためデフォルト値(日単位想定)スケーリング 24*60/5=288
scale = 10

In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
from fin_app_models.model.structured_base.regression import (
    LinearRegression,
    RidgeRegression,
    LassoRegression,
    LGBMRegression,
    CatBoostRegression,
)
from fin_app_models.model.structured_base.timeseries_model.dnn.lstm import BILSTMRegression
from fin_app_models.processing import (
    IStructuredDataProcessing,
    DefaultStructuredDataProcessing
)

In [11]:
from sklearn.ensemble import RandomForestRegressor

In [12]:
import matplotlib.pyplot as plt

In [13]:
df_feats_buy = create_single_ts_features(
    df_btc_return['price_buy'],
    macd_fastperiod=scale*12,
    macd_slowperiod=scale*26,
    macd_signalperiod=scale*9,
    bb_periods=[scale*7, scale*20, scale*30, scale*60],
    basic_stats_period=scale*14,
    atr_period=scale*14,
    return_lags=[scale*1, scale*3, scale*7, scale*10, scale*20, scale*30, scale*60],
    col_name_prefix='buy',
)
df_feats_sell = create_single_ts_features(
    df_btc_return['price_sell'],
    macd_fastperiod=scale*12,
    macd_slowperiod=scale*26,
    macd_signalperiod=scale*9,
    bb_periods=[scale*7, scale*20, scale*30, scale*60],
    basic_stats_period=scale*14,
    atr_period=scale*14,
    return_lags=[scale*1, scale*3, scale*7, scale*10, scale*20, scale*30, scale*60],
    col_name_prefix='sell',
)
df_feats_spread = create_single_ts_features(
    (df_btc_return['price_sell']-df_btc_return['price_buy']).to_frame('price_spread')['price_spread'],
    macd_fastperiod=scale*12,
    macd_slowperiod=scale*26,
    macd_signalperiod=scale*9,
    bb_periods=[scale*7, scale*20, scale*30, scale*60],
    basic_stats_period=scale*14,
    atr_period=scale*14,
    return_lags=[scale*1, scale*3, scale*7, scale*10, scale*20, scale*30, scale*60],
    col_name_prefix='spread',
)
df_feats_size_buy = create_single_ts_features(
    df_btc_return['size_buy'],
    macd_fastperiod=scale*12,
    macd_slowperiod=scale*26,
    macd_signalperiod=scale*9,
    bb_periods=[scale*7, scale*20, scale*30, scale*60],
    basic_stats_period=scale*14,
    atr_period=scale*14,
    return_lags=[scale*1, scale*3, scale*7, scale*10, scale*20, scale*30, scale*60],
    col_name_prefix='size_buy',
)
df_feats_size_sell = create_single_ts_features(
    df_btc_return['size_sell'],
    macd_fastperiod=scale*12,
    macd_slowperiod=scale*26,
    macd_signalperiod=scale*9,
    bb_periods=[scale*7, scale*20, scale*30, scale*60],
    basic_stats_period=scale*14,
    atr_period=scale*14,
    return_lags=[scale*1, scale*3, scale*7, scale*10, scale*20, scale*30, scale*60],
    col_name_prefix='size_sell',
)
df_feats = pd.concat(
    [df_feats_buy, df_feats_sell, df_feats_spread, df_feats_size_buy, df_feats_size_sell],
    axis=1
)

df_feats.dropna(inplace=True)
FEAT_COLS = df_feats.columns.tolist()

df_feats_target = df_feats.merge(df_btc_return, how='inner', left_index=True, right_index=True)
df_feats_target = df_feats_target.clip(-100000000000000, 100000000000000)

df_feats_target_train = df_feats_target[df_feats_target.index < '2021-06-01']
df_feats_target_test = df_feats_target[df_feats_target.index >= '2021-06-01']

ss = StandardScaler()
feats_target_train = ss.fit_transform(df_feats_target_train)
df_feats_target_train = pd.DataFrame(
    data=feats_target_train,
    columns=df_feats_target_train.columns,
    index=df_feats_target_train.index
)
feats_target_test = ss.transform(df_feats_target_test)
df_feats_target_test = pd.DataFrame(
    data=feats_target_test,
    columns=df_feats_target_test.columns,
    index=df_feats_target_test.index
)

In [14]:
df_train_sampled = df_feats_target_train.sample(10000)

In [15]:
bilstm_model = BILSTMRegression(data_processors=[DefaultStructuredDataProcessing()])

In [16]:
bilstm_model.train(
    y_train=df_train_sampled['return_24h'],
    X_train=df_train_sampled[FEAT_COLS],
    seq_len=100,
    n_epoch=50,
)

DEBUG : 2022-01-06 06:09:21,625 : [BILSTMRegression] Start tuning parameters
DEBUG : 2022-01-06 06:09:21,633 : [BILSTMRegression] Done tuning parameters
DEBUG : 2022-01-06 06:09:21,636 : [BILSTMRegression] best_pramas : {}


In [17]:
# preds_train = bilstm_model.predict(X=df_feats_target_train[FEAT_COLS])
preds_train = bilstm_model.predict(X=df_train_sampled[FEAT_COLS])

In [18]:
preds_train.shape

(9900,)

In [19]:
((preds_train.flatten() * df_train_sampled['return_24h'].to_numpy()[-len(preds_train.flatten()):]) > 0).sum() / len(preds_train.flatten())

0.47484848484848485

In [ ]:
((preds_train.flatten() * df_feats_target_train['return_24h'].to_numpy()[-len(preds_train.flatten()):]) > 0).sum() / len(preds_train.flatten())

In [ ]:
preds_test = bilstm_model.predict(X=df_feats_target_test[FEAT_COLS])

In [ ]:
((preds_test.flatten() * df_feats_target_test['return_24h'].to_numpy()[-len(preds_test.flatten()):]) > 0).sum() / len(preds_test.flatten())

In [ ]:
df_train_sampled = df_feats_target_train.sample(1000)

for target_col in TARGET_COLS:
    rf_model = RandomForestRegressor()
    rf_model.fit(
        df_train_sampled[FEAT_COLS],
        df_train_sampled[target_col],
    )
    pred_train = rf_model.predict(df_feats_target_train[FEAT_COLS])
    pred_test = rf_model.predict(df_feats_target_test[FEAT_COLS])
    train_acc = sum(pred_train * df_feats_target_train[target_col].to_numpy() >= 0) / len(pred_train)
    test_acc = sum(pred_test * df_feats_target_test[target_col].to_numpy() >= 0) / len(pred_test)
    print(f'{target_col}, train_acc : {train_acc}, test_acc : {test_acc}')

return_1h, train_acc : 0.56169177838415, test_acc : 0.4933422103861518
return_3h, train_acc : 0.5803179499347491, test_acc : 0.4940079893475366
return_6h, train_acc : 0.6044014711116384, test_acc : 0.5099866844207723
return_12h, train_acc : 0.662296832364456, test_acc : 0.5033288948069241
return_18h, train_acc : 0.7020998932257682, test_acc : 0.5109853528628495
return_24h, train_acc : 0.7107011507889429, test_acc : 0.5013315579227696
return_48h, train_acc : 0.7630205243801162, test_acc : 0.47203728362183756
return_72h, train_acc : 0.7879345118044845, test_acc : 0.5352862849533955
return_120h, train_acc : 0.8199074623324237, test_acc : 0.4607190412782956
return_168h, train_acc : 0.8494483331356033, test_acc : 0.5256324900133156
return_720h, train_acc : 0.9358168228734132, test_acc : 0.3725033288948069
return_1440h, train_acc : 0.9835686321034524, test_acc : 0.16511318242343542
return_2160h, train_acc : 0.9925258037726895, test_acc : 0.39580559254327563


In [ ]:
import numpy as np

In [ ]:
len(FEAT_COLS)

215

In [ ]:
df_train_sampled = df_feats_target_train.sample(10000)

for target_col in TARGET_COLS:
    rf_model = RandomForestRegressor()
    rf_model.fit(
        df_train_sampled[FEAT_COLS],
        df_train_sampled[target_col],
    )
    pred_train = rf_model.predict(df_feats_target_train[FEAT_COLS])
    pred_test = rf_model.predict(df_feats_target_test[FEAT_COLS])
    train_acc = sum(pred_train * df_feats_target_train[target_col].to_numpy() >= 0) / len(pred_train)
    test_acc = sum(pred_test * df_feats_target_test[target_col].to_numpy() >= 0) / len(pred_test)
    thresh_test_accs = {}
    for thresh in [0.1, 0.2, 0.4, 0.7, 1.0, 2.0]:
        filter_indices = np.where(np.abs(pred_test) > thresh)
        if len(pred_test[filter_indices]) == 0:
            continue
        test_acc_thresh = sum(pred_test[filter_indices] * df_feats_target_test[target_col].to_numpy()[filter_indices] >= 0) / len(pred_test[filter_indices])
        thresh_test_accs[f'test_acc_th{thresh}'] = test_acc_thresh
    print(thresh_test_accs)
    print(f'{target_col}, train_acc : {train_acc}, test_acc : {test_acc}')

{'test_acc_th0.1': 0.4928, 'test_acc_th0.2': 0.46560319042871384, 'test_acc_th0.4': 0.4847161572052402, 'test_acc_th0.7': 0.5517241379310345, 'test_acc_th1.0': 0.625}
return_1h, train_acc : 0.8232293273223396, test_acc : 0.49567243675099865
{'test_acc_th0.1': 0.47879684418145957, 'test_acc_th0.2': 0.4816204051012753, 'test_acc_th0.4': 0.4901639344262295, 'test_acc_th0.7': 0.5561497326203209, 'test_acc_th1.0': 0.6538461538461539}
return_3h, train_acc : 0.882726302052438, test_acc : 0.47969374167776296
{'test_acc_th0.1': 0.5012531328320802, 'test_acc_th0.2': 0.5141471301535974, 'test_acc_th0.4': 0.5011764705882353, 'test_acc_th0.7': 0.4852941176470588, 'test_acc_th1.0': 0.625}
return_6h, train_acc : 0.9162415470399811, test_acc : 0.4933422103861518
{'test_acc_th0.1': 0.5226883561643836, 'test_acc_th0.2': 0.5324211778703153, 'test_acc_th0.4': 0.5380774032459426, 'test_acc_th0.7': 0.5963302752293578, 'test_acc_th1.0': 0.6410256410256411}
return_12h, train_acc : 0.937418436350694, test_acc 

In [ ]:
train_acc

0.8801755842923241

In [ ]:
df_feats_target_test.head()

,buy_price_buy,buy_rsi,buy_macd,buy_bb70_high,buy_bb70_mid,buy_bb70_low,buy_bb70_std,buy_bb200_high,buy_bb200_mid,buy_bb200_low,buy_bb200_std,buy_bb300_high,buy_bb300_mid,buy_bb300_low,buy_bb300_std,buy_bb600_high,buy_bb600_mid,buy_bb600_low,buy_bb600_std,buy_max_140,buy_min_140,buy_median_140,buy_skew_140,buy_kurt_140,buy_ppo_12_26,buy_apo_12_26,buy_cmo_14,buy_return_lag10,buy_return_lag30,buy_return_lag70,buy_return_lag100,buy_return_lag200,buy_return_lag300,buy_return_lag600,buy_momentum_10_30,buy_momentum_10_70,buy_momentum_10_100,buy_momentum_10_200,buy_momentum_10_300,buy_momentum_10_600,...,size_sell_min_140,size_sell_median_140,size_sell_skew_140,size_sell_kurt_140,size_sell_ppo_12_26,size_sell_apo_12_26,size_sell_cmo_14,size_sell_return_lag10,size_sell_return_lag30,size_sell_return_lag70,size_sell_return_lag100,size_sell_return_lag200,size_sell_return_lag300,size_sell_return_lag600,size_sell_momentum_10_30,size_sell_momentum_10_70,size_sell_momentum_10_100,size_sell_momentum_10_200,size_sell_momentum_10_300,size_sell_momentum_10_600,size_sell_weekday,size_sell_year,size_sell_month,size_buy,price_buy,size_sell,price_sell,return_1h,return_3h,return_6h,return_12h,return_18h,return_24h,return_48h,return_72h,return_120h,return_168h,return_720h,return_1440h,return_2160h
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-06-01 11:30:00,1.223795,1.135448,-0.093885,1.230104,1.220633,1.210439,1.242172,1.227168,1.240080,1.252774,0.574159,1.206979,1.241784,1.277791,0.189233,1.190774,1.205125,1.218602,0.723515,1.249727,1.246329,1.235444,0.550098,-0.293135,0.772919,1.032064,1.135448,0.671723,0.811496,-0.721696,-0.345676,-0.125942,-0.194294,0.193054,-0.293868,-0.979712,-0.798500,-0.712806,-0.712069,-0.651249,...,-0.388464,-1.128833,3.138299,3.726160,-0.346258,-0.023971,-0.128546,0.429317,-1.122016,0.179257,1.317842,-0.125709,0.003061,-0.468630,-0.866629,-0.427056,-0.314128,-0.444194,-0.435309,-0.439985,-1.031867,1.757208,0.289789,-0.278318,1.223795,-0.434798,1.224913,-0.905895,0.279475,-1.151928,0.008484,-0.159833,0.280412,1.016951,-0.174276,-0.602489,-1.387565,-0.814318,-0.269007,-0.206750
2021-06-01 11:45:00,1.222841,0.854142,-0.091342,1.224691,1.218956,1.212627,1.087113,1.227167,1.240064,1.252742,0.574533,1.206923,1.241676,1.277627,0.190266,1.191030,1.205373,1.218840,0.723831,1.249727,1.246329,1.234930,0.646004,-0.193193,0.719557,0.963603,0.854142,0.666902,0.890638,-0.941810,-0.416296,-0.155037,-0.330413,0.150580,-0.236728,-1.055251,-0.814165,-0.713933,-0.730316,-0.651423,...,-0.388464,-1.091525,3.142397,3.731985,-0.700119,-0.062905,0.260837,0.836542,0.087106,0.569445,0.084979,0.010962,0.160000,0.838893,-0.892394,-0.799755,-0.861703,-0.852901,-0.843193,-0.828829,-1.031867,1.757208,0.289789,-0.128106,1.222841,-0.377256,1.223772,-0.935872,0.287949,-0.875805,-0.022682,0.089506,0.388762,1.041242,-0.253448,-0.585930,-1.340584,-0.801575,-0.268795,-0.199326
2021-06-01 16:15:00,1.194462,-0.665050,-0.321440,1.217274,1.213539,1.209282,1.000903,1.227716,1.232841,1.237290,0.768712,1.209240,1.237288,1.266041,0.329664,1.193979,1.208688,1.222556,0.720828,1.219210,1.230891,1.215124,0.433922,-0.522095,-1.788265,-2.338366,-0.665050,-0.267803,-2.134677,-0.427749,-0.804965,-1.085208,-0.574591,0.126919,-1.055903,0.128370,0.042401,0.047886,0.173134,0.283692,...,-0.388464,-1.240914,3.314253,3.962144,-0.564163,-0.040468,-0.009610,0.463297,-0.368023,-0.289473,0.698769,0.251294,-0.492947,-0.468362,-0.640027,-0.531889,-0.411714,-0.460254,-0.485949,-0.474201,-1.031867,1.757208,0.289789,-0.418075,1.194462,-0.435275,1.195791,-1.208210,-0.450674,0.371764,0.348877,0.743194,1.287835,1.252273,0.089332,-0.355446,-1.457771,-0.791159,-0.245428,-0.169041
2021-06-01 19:15:00,1.182133,-0.793825,-0.644904,1.214440,1.203730,1.192258,1.281444,1.203197,1.217819,1.232342,0.514020,1.209541,1.230588,1.251825,0.473644,1.193516,1.210605,1.227114,0.685986,1.212801,1.225494,1.200744,1.116011,0.271394,0.590675,0.786294,-0.793825,-0.6796